# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 2:37PM,237899,18,HH-34491,Hush Hush,Released
1,May 23 2022 2:37PM,237899,18,HH-34492,Hush Hush,Released
2,May 23 2022 2:37PM,237899,18,HH-34494,Hush Hush,Released
3,May 23 2022 2:37PM,237899,18,HH-34495,Hush Hush,Released
4,May 23 2022 2:37PM,237899,18,HH-34496,Hush Hush,Released
5,May 23 2022 2:36PM,237898,18,HH-34493,Hush Hush,Released
6,May 23 2022 2:35PM,237701,18,HH6058.1,Hush Hush,Released
7,May 23 2022 2:33PM,237900,21,26694094,"NBTY Global, Inc.",Executing
8,May 23 2022 2:27PM,237897,20,SSNA-2173624880,Sartorius Stedim North America,Released
9,May 23 2022 2:24PM,237896,21,615540,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,237896,Released,1
44,237897,Released,1
45,237898,Released,1
46,237899,Released,5
47,237900,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237896,NaN,NaN,1.0
237897,NaN,NaN,1.0
237898,NaN,NaN,1.0
237899,NaN,NaN,5.0
237900,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0.0,0.0,1.0
237701,0.0,0.0,1.0
237794,3.0,13.0,10.0
237804,12.0,7.0,0.0
237808,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0,0,1
237701,0,0,1
237794,3,13,10
237804,12,7,0
237808,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,0,0,1
1,237701,0,0,1
2,237794,3,13,10
3,237804,12,7,0
4,237808,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,,,1
1,237701,,,1
2,237794,3,13,10
3,237804,12,7,
4,237808,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 2:37PM,237899,18,Hush Hush
5,May 23 2022 2:36PM,237898,18,Hush Hush
6,May 23 2022 2:35PM,237701,18,Hush Hush
7,May 23 2022 2:33PM,237900,21,"NBTY Global, Inc."
8,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America
9,May 23 2022 2:24PM,237896,21,"NBTY Global, Inc."
10,May 23 2022 2:23PM,237895,21,"NBTY Global, Inc."
11,May 23 2022 2:23PM,237894,10,Emerginnova
14,May 23 2022 2:13PM,237892,20,Sartorius Stedim North America
15,May 23 2022 1:53PM,237883,12,Twinlab Consolidated Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 2:37PM,237899,18,Hush Hush,,,5
1,May 23 2022 2:36PM,237898,18,Hush Hush,,,1
2,May 23 2022 2:35PM,237701,18,Hush Hush,,,1
3,May 23 2022 2:33PM,237900,21,"NBTY Global, Inc.",,1,
4,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America,,,1
5,May 23 2022 2:24PM,237896,21,"NBTY Global, Inc.",,,1
6,May 23 2022 2:23PM,237895,21,"NBTY Global, Inc.",,,1
7,May 23 2022 2:23PM,237894,10,Emerginnova,,,3
8,May 23 2022 2:13PM,237892,20,Sartorius Stedim North America,,,1
9,May 23 2022 1:53PM,237883,12,Twinlab Consolidated Corporation,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 2:37PM,237899,18,Hush Hush,5,,
1,May 23 2022 2:36PM,237898,18,Hush Hush,1,,
2,May 23 2022 2:35PM,237701,18,Hush Hush,1,,
3,May 23 2022 2:33PM,237900,21,"NBTY Global, Inc.",,1,
4,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America,1,,
5,May 23 2022 2:24PM,237896,21,"NBTY Global, Inc.",1,,
6,May 23 2022 2:23PM,237895,21,"NBTY Global, Inc.",1,,
7,May 23 2022 2:23PM,237894,10,Emerginnova,3,,
8,May 23 2022 2:13PM,237892,20,Sartorius Stedim North America,1,,
9,May 23 2022 1:53PM,237883,12,Twinlab Consolidated Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 2:37PM,237899,18,Hush Hush,5,,
1,May 23 2022 2:36PM,237898,18,Hush Hush,1,,
2,May 23 2022 2:35PM,237701,18,Hush Hush,1,,
3,May 23 2022 2:33PM,237900,21,"NBTY Global, Inc.",,1,
4,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 2:37PM,237899,18,Hush Hush,5.0,NaN,NaN
1,May 23 2022 2:36PM,237898,18,Hush Hush,1.0,NaN,NaN
2,May 23 2022 2:35PM,237701,18,Hush Hush,1.0,NaN,NaN
3,May 23 2022 2:33PM,237900,21,"NBTY Global, Inc.",NaN,1.0,NaN
4,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 2:37PM,237899,18,Hush Hush,5.0,0.0,0.0
1,May 23 2022 2:36PM,237898,18,Hush Hush,1.0,0.0,0.0
2,May 23 2022 2:35PM,237701,18,Hush Hush,1.0,0.0,0.0
3,May 23 2022 2:33PM,237900,21,"NBTY Global, Inc.",0.0,1.0,0.0
4,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2378505,188.0,37.0,12.0
12,713583,3.0,1.0,0.0
15,237794,10.0,13.0,3.0
16,475771,2.0,0.0,0.0
18,1902448,12.0,0.0,0.0
19,951522,24.0,0.0,0.0
20,1665120,76.0,12.0,17.0
21,713691,2.0,1.0,0.0
22,475733,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2378505,188.0,37.0,12.0
1,12,713583,3.0,1.0,0.0
2,15,237794,10.0,13.0,3.0
3,16,475771,2.0,0.0,0.0
4,18,1902448,12.0,0.0,0.0
5,19,951522,24.0,0.0,0.0
6,20,1665120,76.0,12.0,17.0
7,21,713691,2.0,1.0,0.0
8,22,475733,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,188.0,37.0,12.0
1,12,3.0,1.0,0.0
2,15,10.0,13.0,3.0
3,16,2.0,0.0,0.0
4,18,12.0,0.0,0.0
5,19,24.0,0.0,0.0
6,20,76.0,12.0,17.0
7,21,2.0,1.0,0.0
8,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,188.0
1,12,Released,3.0
2,15,Released,10.0
3,16,Released,2.0
4,18,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,12.0,0.0,3.0,0.0,0.0,0.0,17.0,0.0,0.0
Executing,37.0,1.0,13.0,0.0,0.0,0.0,12.0,1.0,0.0
Released,188.0,3.0,10.0,2.0,12.0,24.0,76.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,12.0,0.0,3.0,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,37.0,1.0,13.0,0.0,0.0,0.0,12.0,1.0,0.0
2,Released,188.0,3.0,10.0,2.0,12.0,24.0,76.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,12.0,0.0,3.0,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,37.0,1.0,13.0,0.0,0.0,0.0,12.0,1.0,0.0
2,Released,188.0,3.0,10.0,2.0,12.0,24.0,76.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()